In [111]:
import pandas as pd
import datetime as dt
import seaborn as sns
import json
import h5py

from pathlib import Path

from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/Colab Notebooks/April/')

print("Importing Complete")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Importing Complete


In [112]:
#Let's take a look at how many tickers were on yahoo finance and those that were missing
#We will also need all the SP500 changes to check if our historicals match up with the time they were in the SP500

avaliable_yf_tickers_filepath = '/p3data/logs/avaliable_yf_tickers.json'
avaliable_yf_tickers_filepath = '/content/drive/MyDrive/Colab Notebooks/April/data/yf/logs/avaliable_yf_tickers.json'

missing_yf_tickers_filepath = '/p3data/logs/missing_yf_tickers.json'
missing_yf_tickers_filepath = '/content/drive/MyDrive/Colab Notebooks/April/data/yf/logs/missing_yf_tickers.json'

sp500_changes_filepath = 'p3data/S&P500 Consitutents 20061009-20220116.json'
sp500_changes_filepath = '/content/drive/MyDrive/Colab Notebooks/April/data/S&P500 Consitutents 20061009-20220116.json'

with open(avaliable_yf_tickers_filepath, 'r') as f:
  avaliable_yf_tickers = json.load(f)

with open(missing_yf_tickers_filepath, 'r') as f:
  missing_yf_tickers = json.load(f)

sp500_changes = pd.read_json(sp500_changes_filepath) #Reminder that we saved the sp500_changes with pd.to_json in part 2 so we can load with pd.read_json() here

print('You have {} avaliable tickers and {} missing tickers for a total of {} tickers'.format(len(avaliable_yf_tickers),
                                                                                              len(missing_yf_tickers),
                                                                                              len(avaliable_yf_tickers+missing_yf_tickers)))

You have 672 avaliable tickers and 177 missing tickers for a total of 849 tickers


In [6]:
#Reminder to handle this
#avaliable_yf_tickers.remove('PGL')
#avaliable_yf_tickers.remove('UVN')

In [113]:
#Let's also load the historicals we recently downloaded from Yahoo Finance to memory
#I am loading them from hdf5 files,  but the csv loader is avaliable at the end of tutorial 2

def load_hdf5_tickers_as_pd_historicals(tickers, filepath):
  historicals = dict()
  columns = ['Date', 'Open', 'High', 'Low', 'Close', 'Volume']
  
  for ticker in tickers:
    hdf5_filepath = f'{filepath}/{ticker}.hdf5'
    ticker_file = Path(hdf5_filepath)
    if ticker_file.is_file():
      with h5py.File(hdf5_filepath, 'r') as f:
        group = f['historicals']
        data = group['15Y'][()]
      dataset = pd.DataFrame(data=data, columns=columns)
      dataset['Date'] = pd.to_datetime(dataset['Date'], unit='s') #Change our timestamps back to datetimes
      dataset = dataset.set_index('Date')
      historicals[ticker] = dataset
    else:
      print('Error {} ticker is missing'.format(ticker))
  print('All Historicals Have Been Saved to Memory')
  return historicals

historicals_filepath = 'p3data/historicals'
historicals_filepath = '/content/drive/MyDrive/Colab Notebooks/April/data/yf'
historicals = load_hdf5_tickers_as_pd_historicals(avaliable_yf_tickers, historicals_filepath)

Error PGL ticker is missing
Error UVN ticker is missing
All Historicals Have Been Saved to Memory


In [115]:
#Now let's look at the data we have

def count_all_historicals_lengths(historicals):
  hist_len_counter = dict()
  for ticker, historical in historicals.items():
    hist_len_counter.setdefault(len(historical), 0)
    hist_len_counter[len(historical)] += 1
  return hist_len_counter

hist_len_counter = count_all_historicals_lengths(historicals)

#Give them a quick sort by most common lengths and highest lenths
most_common_lens = sorted(hist_len_counter.items(), key=lambda x:x[1], reverse=True)
highest_lens = sorted(hist_len_counter.items(), key=lambda x:x[0], reverse=True)

print('Most Common Lens:\n {}'.format(most_common_lens))
print('\nHighest Lens:\n {}'.format(highest_lens))

Most Common Lens:
 [(3777, 515), (464, 2), (1882, 2), (3677, 2), (3778, 2), (1649, 2), (1954, 2), (2163, 2), (2279, 1), (423, 1), (1008, 1), (2988, 1), (2057, 1), (1061, 1), (2437, 1), (2945, 1), (1920, 1), (2559, 1), (455, 1), (3136, 1), (3461, 1), (493, 1), (1137, 1), (3118, 1), (2734, 1), (3735, 1), (2367, 1), (3161, 1), (2978, 1), (2760, 1), (1149, 1), (2921, 1), (941, 1), (2157, 1), (1844, 1), (238, 1), (3032, 1), (2809, 1), (2080, 1), (2167, 1), (2540, 1), (1154, 1), (598, 1), (670, 1), (138, 1), (3696, 1), (1846, 1), (3706, 1), (1366, 1), (3066, 1), (3358, 1), (392, 1), (716, 1), (338, 1), (2468, 1), (2504, 1), (823, 1), (3002, 1), (1704, 1), (1299, 1), (2332, 1), (2434, 1), (2603, 1), (1559, 1), (2793, 1), (721, 1), (722, 1), (2797, 1), (3063, 1), (1397, 1), (2811, 1), (3006, 1), (3072, 1), (2735, 1), (28, 1), (529, 1), (2727, 1), (2040, 1), (502, 1), (840, 1), (1575, 1), (1313, 1), (1911, 1), (2191, 1), (1181, 1), (73, 1), (3451, 1), (1826, 1), (2753, 1), (3754, 1), (1689, 1),

In [116]:
#From above we see that the highest lens is 3778 while the majority of the historicals (515 of them) sit at a comfortable 3777 length
#Additionally in "Highest Lens", we see that past the length of 3777, very few tickers have the same amount of missing tickers
#Meaning that 3777 is the typical length of 15 years of data and we will use this length for reference and comparison
#However, the 3778 length does stand out, let's take a look at which tickers are causing this

def look_for_target_historicals_lengths(historicals, target_len):
  target_len_tickers = [ticker
                        for ticker, historical in historicals.items()
                        if target_len == len(historical)]
  return target_len_tickers

outlier_len = 3778
outlier_len_tickers = look_for_target_historicals_lengths(historicals, outlier_len)
print(outlier_len_tickers) #We see that GS stands out. Since this is a single ticker, we will simply cut the data point later that is extra from the dataset to keep the data uniform

standard_len = 3777
standard_len_tickers = look_for_target_historicals_lengths(historicals, standard_len)
print(standard_len_tickers)

['DIA', 'GS']
['A', 'AAL', 'AAP', 'AAPL', 'ABC', 'ABMD', 'ABT', 'ACN', 'ADBE', 'ADI', 'ADM', 'ADP', 'ADS', 'ADSK', 'AEE', 'AEP', 'AES', 'AFL', 'AIG', 'AIV', 'AIZ', 'AJG', 'AKAM', 'ALB', 'ALGN', 'ALK', 'ALL', 'AMAT', 'AMD', 'AME', 'AMG', 'AMGN', 'AMP', 'AMT', 'AMZN', 'AN', 'ANF', 'ANSS', 'ANTM', 'AON', 'AOS', 'APA', 'APD', 'APH', 'ARE', 'ASH', 'ATGE', 'ATI', 'ATO', 'ATVI', 'AVB', 'AVY', 'AXP', 'AYI', 'AZO', 'BA', 'BAC', 'BAX', 'BBBY', 'BBWI', 'BBY', 'BC', 'BDX', 'BEN', 'BIG', 'BIIB', 'BIO', 'BK', 'BKNG', 'BKR', 'BLK', 'BLL', 'BMY', 'BRO', 'BSX', 'BWA', 'BXP', 'C', 'CAG', 'CAH', 'CAT', 'CB', 'CBRE', 'CCEP', 'CCI', 'CCL', 'CCU', 'CDNS', 'CE', 'CERN', 'CF', 'CHD', 'CHRW', 'CI', 'CIEN', 'CINF', 'CL', 'CLF', 'CLX', 'CMA', 'CMCSA', 'CME', 'CMG', 'CMI', 'CMS', 'CNC', 'CNP', 'CNX', 'COF', 'COO', 'COP', 'COST', 'CPB', 'CPRT', 'CRL', 'CRM', 'CSCO', 'CSX', 'CTAS', 'CTRA', 'CTSH', 'CTXS', 'CVS', 'CVX', 'D', 'DD', 'DDS', 'DE', 'DGX', 'DHI', 'DHR', 'DIS', 'DISCA', 'DISH', 'DLR', 'DLTR', 'DOV', 'DPZ',

In [117]:
#Let's make sure all the date ranges at 3777 is the same
#From above we see that AAPL has all this data, let's just do a quick comparison that all the dates of the other tickers with the same length

def check_for_uniformity_of_dates(historicals, 
                                tickers_to_compare, 
                                comparsion_ticker):
  for ticker in tickers_to_compare:
    if historicals[ticker].index.equals(historicals[comparison_ticker].index): 
      pass
    else: 
      print('{} does not have the same dates as {}'.format(ticker, comparison_ticker)) #This will print only if dates do not match with AAPL
  print('Comparison Completed')

comparison_ticker = 'AAPL'
check_for_uniformity_of_dates(historicals,
                              standard_len_tickers,
                              comparison_ticker)

Comparison Completed


In [118]:
#Since we have now confirmed that all the tickers that have the length of 3777 are at the same dates, we can now get to work on the missing dates
#Let's grab all the tickers that have missing data

tickers_with_full_data = standard_len_tickers + outlier_len_tickers 
tickers_with_missing_dates = set(historicals) - set(tickers_with_full_data)
print('Tickers with missing dates:\n {}'.format(tickers_with_missing_dates))

Tickers with missing dates:
 {'TWTR', 'EQ', 'NSM', 'NWSA', 'MDP', 'CTX', 'MRNA', 'DELL', 'ENPH', 'TRIP', 'VNT', 'SBL', 'ZTS', 'PM', 'MMI', 'WRK', 'KDP', 'ADCT', 'DISCK', 'NCC', 'CDAY', 'PLL', 'NXPI', 'UA', 'FOX', 'TSLA', 'DG', 'SEDG', 'OGN', 'WLTW', 'COL', 'CBE', 'HET', 'CPRI', 'SUN', 'CARR', 'WB', 'SII', 'VRSK', 'DAL', 'PYPL', 'XL', 'DNB', 'KSU', 'MPC', 'TWX', 'HII', 'NFX', 'DFS', 'BRL', 'SLR', 'IR', 'FCPT', 'SOV', 'LW', 'SYF', 'HCA', 'CMX', 'FB', 'ANET', 'KEYS', 'NE', 'DOW', 'NWS', 'TEK', 'HPE', 'QRVO', 'SVU', 'FOXA', 'PD', 'S', 'ALLE', 'FTNT', 'CFG', 'CHTR', 'ROH', 'IQV', 'AET', 'INFO', 'APTV', 'CSRA', 'OTIS', 'CBH', 'ETSY', 'KMI', 'BOL', 'NOW', 'HPC', 'V', 'SAF', 'BHF', 'CZR', 'BUD', 'CTLT', 'SHLD', 'LIFE', 'DYN', 'TEL', 'CDW', 'ESRX', 'NLSN', 'HWM', 'FLT', 'MSCI', 'AMCR', 'CTVA', 'FTV', 'ARNC', 'FANG', 'HLT', 'BR', 'XYL', 'KHC', 'COTY', 'PX', 'LYB', 'EPAM', 'ABBV', 'PSX', 'CA', 'SCG', 'MON', 'FRC', 'CPWR', 'GNRC', 'JAVA', 'FBHS', 'CVG', 'CBOE', 'H', 'SE', 'ULTA', 'HCP', 'EVHC', 'C

In [119]:
#Compile all the missing dates for each ticker with missing dates we will compare it the full date range that the other tickers have
def compile_all_missing_dates(historicals,
                              tickers_with_missing_dates,
                              full_date_range):
  missing_tickers_and_dates = {ticker: full_date_range.difference(historicals[ticker].index) 
                              for ticker in tickers_with_missing_dates}
  return missing_tickers_and_dates

full_date_range = historicals['AAPL'].index #We already know AAPL has the entire date range, let's get its dates for comparsion

missing_tickers_and_dates = compile_all_missing_dates(historicals,
                                                      tickers_with_missing_dates,
                                                      full_date_range)

print(missing_tickers_and_dates['TWTR']) #Let's double check and take a look at one of our tickers and the dates its missing

DatetimeIndex(['2007-01-22', '2007-01-23', '2007-01-24', '2007-01-25',
               '2007-01-26', '2007-01-29', '2007-01-30', '2007-01-31',
               '2007-02-01', '2007-02-02',
               ...
               '2013-10-24', '2013-10-25', '2013-10-28', '2013-10-29',
               '2013-10-30', '2013-10-31', '2013-11-01', '2013-11-04',
               '2013-11-05', '2013-11-06'],
              dtype='datetime64[ns]', name='Date', length=1713, freq=None)


In [ ]:
#Since we are only looking for the dates when they are in the SP500
#We can filter out the missing dates when they are not in the SP500 as we will not be using those
#We will need the sp500 changes we loaded in earlier
#Additionally we will have to consider if the ticker drops out from the SP500 and re-enters at a later date
#This will require some legwork to get

#Find first date, can try aggrgate
#Then ensure that each next date
#Change entire column to a chained list, then use iloc LOL, which is not a bad idea
#Take a slice

def filter_out_when_dates_not_in_sp500(missing_tickers_and_dates, sp500_changes):
  
  for ticker in missing_tickers_and_dates:


  if ticker in sp500_changes.loc[date,tickers]:


  pass

#All dates it is in the sp500
#Then compare it 

#Using start and end dates and knowing what data we are missing, trim the missing data list to only include the time the ticker was in the sp500
def _trim_dates_outside_of_sp500_date_range(missing_historicals, date_range_for_tickers_in_sp500):
  for ticker, (start_date_in_sp500, end_date_in_sp500) in date_range_for_tickers_in_sp500.items():
    for date in missing_historicals[ticker]:
      if date < start_date_in_sp500 or date > end_date_in_sp500:
        missing_historicals[ticker].remove(date)
        if missing_historicals[ticker] == []:
          missing_historicals.pop(ticker, None)
  return missing_historicals

#Find Start and End Date for the missing tickers for when they were in the SP500
def _find_when_missing_ticker_was_in_sp500(missing_historicals, changes_in_sp500):
  date_range_for_tickers_in_sp500 = {}
  for ticker in missing_historicals:
    date_range_for_tickers_in_sp500[ticker] = [_find_ticker_start_date_in_sp500(ticker, changes_in_sp500), 
                          _find_ticker_end_date_in_sp500(ticker, changes_in_sp500)]
  return date_range_for_tickers_in_sp500

def _find_ticker_start_date_in_sp500(ticker, sp500_changes):
  for date, sp500_constituents in sp500_changes.itertuples():
    if ticker in sp500_constituents:
      start_date = date
      return start_date
      
def _find_ticker_end_date_in_sp500(ticker, sp500_changes):
  for date, sp500_constituents in sp500_changes.iloc[::-1].itertuples():
    if ticker in sp500_constituents:
      end_date = date
      return end_date

In [ ]:
#Let's inspect the missing data and see when we are missing data using seaborn
def count_missing_dates(missing_tickers_and_dates, full_date_range):
  missing_dates_counter = {date: 0 for date in full_date_range}

  for dates in missing_tickers_and_dates.values():
    for date in dates:
      missing_dates_counter[date] += 1
  return missing_dates_counter

missing_dates_and_counter = count_missing_dates(missing_tickers_and_dates, full_date_range)
sns.barplot(data=missing_dates_and_counter)

In [75]:
#Let's check the average size of length of this data
#Now you don't need the full length of each ticker, just the time they were in the SP500
#Additionally, if they have history before they join the SP500, it helpful in order to run moving averages or other sliding window indicators before they join
#But for now it is not neccssary, let's set up the bare minimum requirements and check that all the data is there

#Ok we now need to keep track of these changes
#best way too?